### Master clean code
The code is to clean the following data files.
* all business from Just Jeans, Jay Jays, Jacqui E, Portman’s, Dotti, Peter Alexander, and Smiggle, Target, Kmart and Big W.
* Baby bunting data 71 stores.
* Myer, David jones and other retail data
* toys us data
* Economic data - birth rate
* MYer online fulfilment_to postcode location 

### Importing

In [3]:
#### import
import pandas as pd
import os
import re

### Data 1  - PMV
all business from Just Jeans, Jay Jays, Jacqui E, Portman’s, Dotti, Peter Alexander, and Smiggle, Target, Kmart and Big W.

In [25]:
os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Silver")
## define what columns to use
columns_keep = ['original_StoreName', 'original_Address', 'lat', 'lon',  'postcode', 'district', 'suburb',
    'city',  'state', 'state_code']


batch1 = pd.read_csv('geocoded_by_geoapify-13_04_2023, 7_05_45 pm.csv',usecols=columns_keep)
batch2 = pd.read_csv('geocoded_by_geoapify-13_04_2023, 7_19_08 pm.csv',usecols=columns_keep)
batch3 = pd.read_csv('geocoded_by_geoapify-13_04_2023, 7_24_28 pm.csv',usecols=columns_keep)

## combine all the batches
premier_business = pd.concat([batch1, batch2, batch3], axis=0)

## extract the business name from the original_StoreName the first word before space
premier_business['business_name'] = premier_business['original_StoreName'].str.split(' ').str[0]

## if the business name is "Baby" regardless of the case, then it is "Baby Bunting", else if the business name is "Just" then it is "Just Jeans", else if "Jay" then "Jay Jays"
premier_business['business_name'] = premier_business['business_name'].apply(lambda x: 'Baby Bunting' 
                                    if 'baby' in x.lower() else (
                                    'Just Jeans' if 'just' in x.lower() else (
                                    'Jay Jays' if 'jay' in x.lower() else (
                                    'Jacqui E' if 'jacqui' in x.lower() else (
                                    'Portmans' if 'portmans' in x.lower() else (
                                    'Dotti'    if 'dotti' in x.lower()    else (
                                    'Peter Alexander' if 'peter' in x.lower() else (
                                    'Smiggle' if 'smiggle' in x.lower() else (
                                    'Target' if 'target' in x.lower() else (
                                    'Kmart' if 'mart' in x.lower()  else (
                                    'BigW'  if 'big' in x.lower() else x)))))))))))



## check groupby count
premier_business.groupby('business_name').count()

## only keep those business names relevant
premier_business = premier_business[premier_business['business_name'].isin(
    ['Just Jeans', 'Jay Jays', 'Jacqui E', 'Portmans', 'Dotti', 'Peter Alexander', 'Smiggle', 'Target', 'Kmart', 'BigW'])]

## remove duplicates
premier_business = premier_business.drop_duplicates(subset=['original_StoreName', 'original_Address', 'lat', 'lon',  'postcode', 'district', 'suburb',
    'city', 'state', 'state_code'])

## print the group count by each business name to check
premier_business.groupby('business_name').count()


## check the missing value quality
premier_business.isnull().sum()

## check records with postcode missing
# premier_business[premier_business['postcode'].isnull()]

## check records with postcode missing
# premier_business[premier_business['state'].isnull()]

original_StoreName      0
original_Address        0
lat                     0
lon                     0
postcode               32
district              460
suburb                582
city                   32
state                  18
state_code             18
business_name           0
dtype: int64

#### missing value treatment

In [26]:
#########################missing data treatment ########################################

### treatment for postcode missing
## extract four digit post code from original address, they are located to the left of ", Autralia" from the right hand side
premier_business['postcode_c'] = premier_business['original_Address'].str.split(', Australia').str[0].str.split(' ').str[-1]

## if missing postcode, it will be postcode_c
premier_business['postcode'] = premier_business['postcode'].fillna(premier_business['postcode_c'])


#### treatment for state missing, by hardcoding them as Caemberrra
premier_business['state'] = premier_business['state'].fillna('Canberra')

### treatment for state_code missing 
premier_business['state_code'] = premier_business['state_code'].fillna('ACT')

### treatment for suburb missing
premier_business['suburb_c'] = premier_business.apply(lambda x: x['original_Address'].split(x['state_code'].upper())[0].split(x['state_code'].lower())[0].split()[-1].strip(), axis=1)
premier_business['suburb'] = premier_business['suburb'].fillna(premier_business['suburb_c'])

## check the missing value quality
premier_business.isnull().sum()


original_StoreName      0
original_Address        0
lat                     0
lon                     0
postcode                0
district              460
suburb                  0
city                   32
state                   0
state_code              0
business_name           0
postcode_c              0
suburb_c                0
dtype: int64

### Data 2 Baby Bunting
Baby bunting data 71 stores.

In [27]:
### import the data
os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Bronze")
columns_to_keep = [
'name', 'full_address',  'postal_code', 'city', 'state', 'latitude', 'longitude' ]
BabyBunting = pd.read_excel('Baby bunting.xlsx',usecols=columns_to_keep)

BabyBunting1 = BabyBunting[BabyBunting["name"].str.contains("Baby Bunting")]

## hardcoding business_name as Baby Bunting
BabyBunting1['business_name'] = 'Baby Bunting'

## rename the columns in competitors
new_columns = {'name':"original_StoreName", 
            'full_address': "original_Address", 
            "latitude" : "lat", 
            "longitude" : "lon", 
            "postal_code" : "postcode",
            "city": "suburb" }

BabyBunting1.rename(columns=new_columns, inplace=True)

#### check missing vlaues   --- no missing good
BabyBunting1.isnull().sum()

C:\Users\mkong2\AppData\Local\Temp\ipykernel_8016\2664716138.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BabyBunting1['business_name'] = 'Baby Bunting'
C:\Users\mkong2\AppData\Local\Temp\ipykernel_8016\2664716138.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BabyBunting1.rename(columns=new_columns, inplace=True)


original_StoreName    0
original_Address      0
suburb                0
postcode              0
state                 0
lat                   0
lon                   0
business_name         0
dtype: int64

### Data 3  Myer DJs
Myer, David jones and other retail data


In [28]:
#import competitor

os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Bronze")

columns_to_keep = ['MASTER_STORE_ID', 'COMPETITOR_TYPE_ENGLISH','Locality', "Admin level 1",
    'FORMATTED_ADDRESS', 'LATITUDE',
    'LONGITUDE',  'Postcode', ]

# Read the Excel file with sheet name
sheet_name = 'competitor_stores' # Replace with the name of the sheet you want to read
myer_djs = pd.read_excel('myer-data_v4.xlsx', sheet_name=sheet_name,usecols=columns_to_keep)

## rename the columns in competitors
new_columns = {'COMPETITOR_TYPE_ENGLISH':"business_name", 
            'FORMATTED_ADDRESS': "original_Address", 
            "LATITUDE" : "lat", 
            "LONGITUDE" : "lon", 
            "Postcode" : "postcode",
            "Admin level 1" : "state",
            'Locality' : 'suburb' }

myer_djs.rename(columns=new_columns, inplace=True)

# filling missing value, if missing they are Myers
myer_djs['business_name'].fillna('Myers', inplace=True)

## check groupby count
myer_djs.groupby('business_name').count()

,MASTER_STORE_ID,original_Address,lat,lon,suburb,state,postcode
business_name,,,,,,,
Chemist Warehouse,239,239,239,239,239,239,239
David Jones,39,39,39,39,39,39,39
Debenhams,1,1,1,1,1,1,1
H&M,32,32,32,32,32,32,32
Harris Scarfe,33,33,33,33,33,33,33
Harvey Norman,116,116,116,116,116,116,116
JB Hi-Fi,134,134,134,134,134,134,134
Myers,59,59,59,59,59,59,59
Uniqlo,18,18,18,18,18,18,18


### Data 4 Toys R us
toys r us   44 stores in AUS

In [29]:
os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Bronze")

# Read the Excel file with sheet name

columns_to_keep = ['Store No.', 'Store','City/Town', 'State/Province/Region', 'Postal Code','Country']

sheet_name = 'Store_List' # Replace with the name of the sheet you want to read
Toysrus = pd.read_excel('Project alpha databook.xlsx', sheet_name=sheet_name,usecols=columns_to_keep)
Toysrus_filter = Toysrus[Toysrus['Country'] =='Australia']

## read the second sheet with lat and lon
sheet_name = 'Competitors'
geocode = pd.read_excel('Project alpha databook.xlsx', sheet_name=sheet_name)
geocode_filter = geocode[geocode['Company'] =='ToysRUs']

# Merge the two DataFrames based on store ID
Toysrus1 = Toysrus_filter.merge(geocode_filter, left_on='Store No.', right_on='Store ID') 
print(Toysrus1.shape)

## drop some columns from the dataframe
Toysrus1.drop(['Store ID', 'Company', 'Country'], axis=1, inplace=True)
Toysrus1['business_name'] = 'ToysRUs'

## rename the columns in competitors
new_columns = {'Store No.':"MASTER_STORE_ID", 
            'Store': "original_StoreName", 
            'State/Province/Region' : 'state',
            "Postal Code" : "postcode",
            "Latitude" : "lat", 
            "Long" : "lon", 
            'Address' :'original_Address',
            'City/Town' : 'suburb' }

Toysrus1.rename(columns=new_columns, inplace=True)

#### check missing vlaues   --- no missing good
Toysrus1.isnull().sum()


(44, 11)


MASTER_STORE_ID       0
original_StoreName    0
suburb                0
state                 0
postcode              0
original_Address      0
lat                   0
lon                   0
business_name         0
dtype: int64

### Merging all 4 sets

In [30]:
## append the 4 datasets
all_business = pd.concat([premier_business, myer_djs, BabyBunting1,Toysrus1], ignore_index=True)

## check missing data
all_business.isnull().sum()

## groupby business name count
all_business.groupby('business_name').count()

,original_StoreName,original_Address,lat,lon,postcode,district,suburb,city,state,state_code,postcode_c,suburb_c,MASTER_STORE_ID
business_name,,,,,,,,,,,,,
Baby Bunting,71,71,71,71,71,0,71,0,71,0,0,0,0
BigW,156,156,156,156,156,107,156,150,156,156,156,156,0
Chemist Warehouse,0,239,239,239,239,0,239,0,239,0,0,0,239
David Jones,0,39,39,39,39,0,39,0,39,0,0,0,39
Debenhams,0,1,1,1,1,0,1,0,1,0,0,0,1
Dotti,88,88,88,88,88,52,88,86,88,88,88,88,0
H&M,0,32,32,32,32,0,32,0,32,0,0,0,32
Harris Scarfe,0,33,33,33,33,0,33,0,33,0,0,0,33
Harvey Norman,0,116,116,116,116,0,116,0,116,0,0,0,116


In [31]:
# export all_business to csv
os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Silver")
all_business.to_csv('all_business.csv', index=False)

### Data 5 Shopping Malls
this is all shopping mall data


In [20]:
os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Silver")

# List of columns to keep from the CSV files
columns_keep = ['original_StoreName', 'original_Address', 'lat', 'lon',  'postcode', 'district', 'suburb', 'city',  'state', 'state_code']

# Read in the CSV files as separate DataFrames
batch1 = pd.read_csv('Geocoded_mall1.csv', usecols=columns_keep)
batch2 = pd.read_csv('Geocoded_mall2.csv', usecols=columns_keep)
batch3 = pd.read_csv('Geocoded_mall3.csv', usecols=columns_keep)

# Combine the separate DataFrames into a single DataFrame
ShoppingCenters = pd.concat([batch1, batch2, batch3], axis=0)

# Regular expression to match store number
pattern = r'stores: (\d+)'

# Extract the store number from the original_Address column
store_numbers = ShoppingCenters['original_Address'].str.extract(pattern)

# Rename the resulting column to store_number
store_numbers.columns = ['store_number']

# Add the store_number column to the original DataFrame
ShoppingCenters = pd.concat([ShoppingCenters, store_numbers], axis=1)

# export all_business to csv
os.chdir("C:\\Users\\mkong2\\OneDrive - KPMG\\Documents\\2023 Projects\\Myer Toys R us\\Silver")
ShoppingCenters.to_csv('ShoppingCenters.csv', index=False)


original_StoreName       0
original_Address         0
lat                     27
lon                     27
suburb                1110
district               660
postcode               317
city                    57
state                   29
state_code              29
dtype: int64